# Aurora Forecasting - Part 01: Feature Backfill

🗒️ This notebook is divided into the following sections:
Initialize Hopsworks connection.

Fetch historical Solar Wind & Kp index data using spacepy (OMNI dataset).

Fetch historical Cloud Cover for Stockholm, Luleå, and Kiruna using Open-Meteo.

Create and Insert data into Feature Groups in the Hopsworks Feature Store.

# Imports

In [ ]:
!pip install spacepy cdasws


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import datetime
import hopsworks
from config import HopsworksSettings
import util
import spacepy.omni as omni
import spacepy.time as spt
import spacepy.toolbox as tb
from cdasws import CdasWs
from cdasws.timeinterval import TimeInterval

# Setup settings
settings = HopsworksSettings()

# Login to Hopsworks
project = hopsworks.login(
    project=settings.HOPSWORKS_PROJECT,
    api_key_value=settings.HOPSWORKS_API_KEY.get_secret_value()
)
fs = project.get_feature_store()

2025-12-30 11:53:29,012 WARNING: UserWarning: Qin-Denton/OMNI2 data not found in current format. This module has limited functionality. Run spacepy.toolbox.update(QDomni=True) to download data.

Aurora Project Settings initialized!
2025-12-30 11:53:29,020 INFO: Initializing external client
2025-12-30 11:53:29,021 INFO: Base URL: https://c.app.hopsworks.ai:443
Aurora Project Settings initialized!
2025-12-30 11:53:29,020 INFO: Initializing external client
2025-12-30 11:53:29,021 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-12-30 11:53:29,936 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'

2025-12-30 11:53:29,936 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backe

2025-12-30 11:53:30,944 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1299605

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1299605


# Step 1: Historical Solar Wind & Kp Data (The Label)

We use the OMNI dataset via spacepy to get high-resolution historical satellite data. This includes the magnetic field components and the proton parameters needed to predict the Kp index.

> WARNING

If you want to use the Qin-Denton dataset, you have to change the spacepy configuration. This is caused by a shutted down server from the organization.

Steps to follow:
- locate the configuration folder in home/.spacepy/spacepy.rc
- change "qindenton_url" to https://spp-isois.sr.unh.edu/data_public/omni/omnidata.h5 
- relaunch the kernel and run again

By default the program will download the OMNI2 dataset from the official website, so if you want to use the Qin-Denton dataset, follow the steps above.


In [ ]:
# Update OMNI data (handle errors gracefully)
try:
    tb.update(QDomni=True)
except Exception as e:
    print(f"Warning: Qin-Denton files download failed. Server down: {e}")
    print("Trying with OMNI 2 dataset...")
    try:
        tb.update(omni2=True)
    except Exception as e:
        print(f"OMNI 2 database dowload failed: {e}")
        exit()

Retrieving initial Qin-Denton file ...
Trying with OMNI 2 dataset...
Finding OMNI2 files to download ...
Listing files ...0%Warning: Qin-Denton files download failed. Server down: HTTP Error 404: File not found
Trying with OMNI 2 dataset...
Finding OMNI2 files to download ...
Listing files ...100%
Listing files ...100%
Retrieving OMNI2 files ...
Retrieving OMNI2 files ...
Download Progress ...88%OMNI 2 database dowload failed: Remote end closed connection without response
OMNI 2 database dowload failed: Remote end closed connection without response


: 

In [23]:
# Define time range for backfill (Last 2 years)
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=730)
print("Fetching OMNI data from", start_date, "to", end_date, "\n")

t = spt.Ticktock([start_date, end_date], 'ISO') 
#t = spacepy.time.Ticktock(['2002-02-02T12:00:00', '2002-02-02T12:10:00'], 'ISO')
data = omni.get_omni(t, dbase='OMNI2hourly')

print(data)

Fetching OMNI data from 2023-12-30 19:10:34.643658 to 2025-12-29 19:10:34.643658 

{'AE_index': dmarray([32.82678172,         nan]), 'AL_index': dmarray([-11.23710095,          nan]), 'AU_index': dmarray([21.26597067,         nan]), 'Alfven_mach_number': dmarray([12.9496929,        nan]), 'Ap_index': dmarray([ 4., nan]), 'Bartels_rotation_num': dmarray([2596.,   nan]), 'Bx_GSE': dmarray([3.89711297,        nan]), 'By_GSE': dmarray([-0.46474203,         nan]), 'By_GSM': dmarray([-0.49711303,         nan]), 'Bz_GSE': dmarray([0.12659707,        nan]), 'Bz_GSM': dmarray([0.02659706,        nan]), 'Dst_index': dmarray([11.64742019,         nan]), 'Electric_field_GSM': dmarray([-0.00445332,         nan]), 'Field_mag_ave': dmarray([4.22948397,        nan]), 'Flow_pressure': dmarray([2.52144347,        nan]), 'IMF_spacecraft_ID': dmarray([51., nan]), 'Ion_density': dmarray([11.04103205,         nan]), 'Kp_index': dmarray([10., nan]), 'Lat_angle_ave_field_vector': dmarray([1.93710092,        n

In [20]:
# Create DataFrame
solar_wind_df = pd.DataFrame({
    'time_tag': data['UTC'],
    'by_gsm': data['By_GSM'],
    'bz_gsm': data['Bz_GSM'],
    'density': data['Ion_density'],
    'speed': data['Plasma_bulk_speed'],
    'kp_index': data['Kp_index']
})

print(solar_wind_df)
print(solar_wind_df.shape[0])

                    time_tag    by_gsm    bz_gsm    density       speed  \
0 2023-12-30 18:40:15.576211 -0.648702  0.380305  10.636795  328.170993   
1 2025-12-29 18:40:15.576211       NaN       NaN        NaN         NaN   

   kp_index  
0      10.0  
1       NaN  
2


In [ ]:
# Create DataFrame
solar_wind_df = pd.DataFrame({
    'time_tag': data['UTC'],
    'bx_gsm': data['Bx'],
    'by_gsm': data['By'],
    'bz_gsm': data['Bz'],
    'density': data['n'],
    'speed': data['v'],
    'kp_index': data['Kp']
})

# Data Cleaning: OMNI uses 99.9 or 999.9 as fill values for missing data
solar_wind_df = solar_wind_df[solar_wind_df['kp_index'] < 10] # Kp is 0-9
solar_wind_df = solar_wind_df.dropna()

# Cast time to string/ISO format for Hopsworks compatibility
solar_wind_df['time_tag'] = pd.to_datetime(solar_wind_df['time_tag']).dt.strftime('%Y-%m-%d %H:%M:%S')

print(f"Successfully fetched {len(solar_wind_df)} historical records.")
solar_wind_df.head()

# TEST con nuovo dataset

In [24]:
cdas = CdasWs()

# Get the OMNI dataset
#dataset = 'OMNI_HRO_1MIN'  # 1 minute resolution
dataset = 'OMNI2_H0_MRG1HR'  # 1 hour resolution
var_names = cdas.get_variable_names(dataset)

print('Variable names:', var_names)

# Time interval
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=730)  # 2 years

# cdas.get_data expects a cdasws TimeInterval (or ISO start/end strings).
# Build a TimeInterval from the start and end datetimes to avoid "invalid time0 type".
time_interval = TimeInterval(start_date.strftime('%Y-%m-%dT%H:%M:%S'),
                             end_date.strftime('%Y-%m-%dT%H:%M:%S'))

# Fetch data
status, data = cdas.get_data(dataset, var_names, time_interval)

# Keep this (suggested in the official site)
if 'spacepy' in str(type(data)):
    #  see https://spacepy.github.io/datamodel.html
    print(var_names[0], '=', data[var_names[0]])
    print(data[var_names[0]].attrs)
    for var in var_names:
        print(var, '=', data[var])
        print(data[var].attrs)
        print("\n")
else:
    #  see https://github.com/MAVENSDC/cdflib
    print(var_names[0], '=', data.data_vars[var_names[0]].values)
    print(data.data_vars[var_names[0]].attrs)


Variable names: ['Rot1800', 'IMF1800', 'PLS1800', 'IMF_PTS1800', 'PLS_PTS1800', 'ABS_B1800', 'F1800', 'THETA_AV1800', 'PHI_AV1800', 'BX_GSE1800', 'BY_GSE1800', 'BZ_GSE1800', 'BY_GSM1800', 'BZ_GSM1800', 'SIGMA-ABS_B1800', 'SIGMA-B1800', 'SIGMA-Bx1800', 'SIGMA-By1800', 'SIGMA-Bz1800', 'T1800', 'N1800', 'V1800', 'PHI-V1800', 'THETA-V1800', 'Ratio1800', 'Pressure1800', 'SIGMA-T1800', 'SIGMA-N1800', 'SIGMA-V1800', 'SIGMA-PHI-V1800', 'SIGMA-THETA-V1800', 'SIGMA-ratio1800', 'E1800', 'Beta1800', 'Mach_num1800', 'Mgs_mach_num1800', 'PR-FLX_11800', 'PR-FLX_21800', 'PR-FLX_41800', 'PR-FLX_101800', 'PR-FLX_301800', 'PR-FLX_601800', 'MFLX1800', 'R1800', 'F10_INDEX1800', 'KP1800', 'DST1800', 'AE1800', 'AP_INDEX1800', 'AL_INDEX1800', 'AU_INDEX1800', 'PC_N_INDEX1800', 'Solar_Lyman_alpha1800', 'Proton_QI1800']
Rot1800 = [2596 2596 2596 ... 9999 9999 9999]
{'FIELDNAM': 'Bartels Rotation Number', 'VALIDMIN': 1700, 'VALIDMAX': 9998, 'SCALEMIN': 1700, 'SCALEMAX': 9998, 'FORMAT': 'I4', 'FILLVAL': 9999, 'VAR

In [ ]:
# Extract needed information into a pd frame
mart_ellah = pd.DataFrame({
    'time': data['Epoch'],
    'by_gsm': data['BY_GSM1800'],
    'bz_gsm': data['BZ_GSM1800'],
    'density': data['N1800'],
    'speed': data['V1800'],
    'kp_index': data['KP1800']
})

print(f"Extracted {len(mart_ellah)} records")
mart_ellah.tail(200)

Extracted 17424 records


,time,by_gsm,bz_gsm,density,speed,kp_index
17224,2025-12-18 05:00:00,999.900024,999.900024,2.200000,689.0,27
17225,2025-12-18 06:00:00,999.900024,999.900024,2.200000,688.0,37
17226,2025-12-18 07:00:00,999.900024,999.900024,2.400000,676.0,37
17227,2025-12-18 08:00:00,999.900024,999.900024,2.300000,651.0,37
17228,2025-12-18 09:00:00,999.900024,999.900024,2.300000,638.0,33
...,...,...,...,...,...,...
17419,2025-12-26 08:00:00,999.900024,999.900024,999.900024,9999.0,99
17420,2025-12-26 09:00:00,999.900024,999.900024,999.900024,9999.0,99
17421,2025-12-26 10:00:00,999.900024,999.900024,999.900024,9999.0,99
17422,2025-12-26 11:00:00,999.900024,999.900024,999.900024,9999.0,99


In [27]:
# Data Cleaning: OMNI uses 99.9 or 999.9 as fill values for missing data
mart_ellah = mart_ellah[(mart_ellah['kp_index'] < 99) & 
                        (mart_ellah['by_gsm'] < 999.9) &
                        (mart_ellah['bz_gsm'] < 999.9) &
                        (mart_ellah['density'] < 999.9) &
                        (mart_ellah['speed'] < 9999.9)
                        ]
mart_ellah = mart_ellah.dropna()

print(f"Extracted {len(mart_ellah)} records")
mart_ellah.tail(200)


Extracted 16743 records


,time,by_gsm,bz_gsm,density,speed,kp_index
16828,2025-12-01 17:00:00,1.1,-1.5,2.9,451.0,30
16829,2025-12-01 18:00:00,0.7,-1.1,3.5,439.0,17
16830,2025-12-01 19:00:00,0.9,0.4,3.7,436.0,17
16831,2025-12-01 20:00:00,1.5,0.5,3.5,433.0,17
16832,2025-12-01 21:00:00,2.6,-1.4,3.5,432.0,13
...,...,...,...,...,...,...
17024,2025-12-09 21:00:00,-0.0,-2.6,4.0,351.0,13
17025,2025-12-09 22:00:00,-1.4,-2.2,3.8,347.0,13
17026,2025-12-09 23:00:00,-2.0,-2.1,4.0,345.0,13
17027,2025-12-10 00:00:00,-0.9,-2.2,2.9,335.0,20


# Step 2: Historical City Weather (The Visibility Constraint)

We fetch historical cloud cover for our three target cities. In the final system, the Aurora is only "Visible" if the cloud cover is low.

In [ ]:
weather_backfill_list = []

for city, coords in settings.CITIES.items():
    print(f"Fetching historical cloud cover for {city}...")

    # We use the historical weather function from util.py
    # Modified to fetch 'cloud_cover' specifically
    df_city = util.get_historical_weather(
        city=city,
        start_date=start_date.strftime("%Y-%m-%d"),
        end_date=end_date.strftime("%Y-%m-%d"),
        latitude=coords['lat'],
        longitude=coords['lon']
    )

    # Standardize columns
    df_city['city'] = city

    # Ensure cloud_cover is present
    if 'cloud_cover' not in df_city.columns:
        # Fallback if your util function uses different naming like cloud_cover_mean
        df_city = df_city.rename(columns={'cloud_cover_mean': 'cloud_cover'})

    weather_backfill_list.append(df_city[['city', 'date', 'cloud_cover']])

weather_df = pd.concat(weather_backfill_list)
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.strftime('%Y-%m-%d')
weather_df.head()

🗄️ Step 3: Create Feature Groups and Insert Data

Now we register these datasets in the Hopsworks Feature Store.

In [ ]:
# 1. Create Solar Wind Feature Group
solar_wind_fg = fs.get_or_create_feature_group(
    name="solar_wind_fg",
    version=1,
    primary_key=['time_tag'],
    description="Satellite measurements (Bz, speed, density) and Kp index labels",
    online_enabled=True,
    statistics_config={"enabled": True, "histograms": True, "correlations": True}
)

# 2. Create City Weather Feature Group
city_weather_fg = fs.get_or_create_feature_group(
    name="city_weather_fg",
    version=1,
    primary_key=['city', 'date'],
    description="Historical cloud cover for Stockholm, Luleå, and Kiruna",
    online_enabled=True,
    statistics_config={"enabled": True, "histograms": True}
)

# Insert Data
solar_wind_fg.insert(solar_wind_df)
city_weather_fg.insert(weather_df)

print("Backfill Complete! Data is now in the Hopsworks Feature Store.")